In [15]:
import torch
from torch.autograd import Variable
from torch.functional import F
p1=torch.Tensor([[0,1],[0,1],[1,0]])
l=torch.Tensor([0,1])
l=torch.arange(0,2)
label=Variable(F.one_hot(l,num_classes=5).cuda())

In [16]:
label,l

(tensor([[1, 0, 0, 0, 0],
         [0, 1, 0, 0, 0]], device='cuda:0'),
 tensor([0, 1]))

In [3]:

from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from Dataload.Chaoyang import CHAOYANG
from torchvision.transforms import transforms
from Dataload.patchCamelyon import Camelyonpatch
from model.models import SimSiam
import torch
from torch.autograd import Variable
dataset='chaoyang'
r=0.0
if dataset=='chaoyang':
    backbone='densenet121'
    num_classes=4
    batch_size = 32
    r=0.0
    test_dataset = CHAOYANG(root="/home/hangang/datasets/chaoyang-data",
                            json_name="test.json",
                            train=False,
                            transform1=transforms.Compose([transforms.Resize((256,256)), transforms.ToTensor()]),
                            )
else:
    backbone='densenet121'
    num_classes=2
    batch_size = 128
    test_dataset = Camelyonpatch(train=False,
                                 transform1=transforms.Compose([transforms.ToTensor()]))
    
model_net1 = SimSiam(0.99, backbone, num_classes)
# model_net2 = model_net1
model_net1.cuda()
sd=torch.load(f'/home/hangang/桌面/log/bestchaoyang_0.0_sd(95.97%).pth')
model_net1.load_state_dict(sd)
test_load = DataLoader(test_dataset, batch_size, shuffle=False)

def evaluate(val_load, model_net1_kd):
    print("Evaluate......")
    model_net1_kd.eval()
    # model_net2_kd.eval()
    sum = 0
    num = 0
    TP1 = 0
    FP1 = 0
    FN1 = 0
    TN1 = 0
    conf_matrix1=torch.zeros(4,4)
    
    for x, y, index in tqdm(val_load):
        with torch.no_grad():
            x = Variable(x[0]).cuda()
            y = Variable(y).cuda()
            outputs1 = model_net1_kd.forward_test(x)
            # y_hat2 = model_net2_kd.forward_test(x)
            # y_hat = (y_hat2 + y_hat1) / 2.0
            _, pred1 = torch.max(outputs1.data, 1)
            for p, t in zip(pred1, y):
                conf_matrix1[p, t] += 1
            y_hat = outputs1
            sum += y_hat.shape[0]
            num += torch.sum(y_hat.argmax(dim=1).type(y.dtype) == y)

    test_acc = float(num / sum)

    return test_acc,conf_matrix1

test_acc,conf_matrix = evaluate(test_load, model_net1,
                            # model_net2,
                            )
print(test_acc,conf_matrix)

dim in 1024
dim out 1000
Evaluate......


  0%|          | 0/67 [00:00<?, ?it/s]

0.8597475290298462 tensor([[622.,  83.,   6.,  19.],
        [ 55., 199.,   6.,  33.],
        [ 11.,  13., 824.,  27.],
        [ 17.,  26.,   4., 194.]])


In [5]:
pre=((622)/(622+55+11+17)+(199)/(199+83+13+26)+(824)/(16+824)+(194)/(194+27+33+19))/4.0
pre

0.7984455724563102

In [6]:
Roc=((622)/(622+83+6+19)+(199)/(55+199+6+33)+(824)/(11+13+824+27)+(194)/(17+26+4+194))/4.0
Roc

0.8194823051797202

In [7]:
F1=(2*pre*Roc)/(pre+Roc)
F1

0.808827176194108